In [1]:
import pandas as pd
from collections import Counter

In [40]:
indu_dict = {
'农、林、牧、渔业': 1,
'建筑业': 2,
'电力、热力、燃气及水生产和供应业': 2,
'采矿业': 2,
'制造业': 2,
'居民服务、修理和其他服务业': 3,
'公共管理、社会保障和社会组织': 3,
'交通运输、仓储和邮政业': 3,
'信息传输、软件和信息技术服务业': 3,
'住宿和餐饮业': 3,
'租赁和商务服务业': 3,
'教育': 3,
'金融业': 3,
'科学研究和技术服务业': 3,
'房地产业': 3,
'批发和零售业': 3,
'文化、体育和娱乐业': 3,
'卫生和社会工作': 3,
'水利、环境和公共设施管理业': 3
}

stat_existence = ['开业', '成立', '在业', '存续', '在营', '正常']
stat_exit = ['注销', '吊销', '撤销']

# Chinese Company Registration Status Terms Glossary
#ref: https://www.chinacheckup.com/blogs/articles/chinese-company-registration-status-terms-glossary

In [41]:
for filename in range(2005, 2016):
    print filename
    rst = {}
    dt = pd.read_csv('firm_' + str(filename) + '.csv', sep = '\t')
    N = len(dt)
    for i in range(N):
        stat = str(dt.iloc[i,-8])
        esta_year = dt.iloc[i,-7][:4]
        cate = str(dt.iloc[i,-5])
        lat = float(dt.iloc[i,-2])
        lon = float(dt.iloc[i,-1])
        
        if any(s in stat for s in stat_existence):
            oper_dummy = 1
        elif any(s in stat for s in stat_exit):
            oper_dummy = 0
        else:
            oper_dummy = 'other'
            
        if cate in indu_dict:
            indu_code = indu_dict[cate]
        else:
            indu_code = 'NaN'
            
        if (indu_code == 'NaN'):
            continue
            
        geo_index = str(round(lon, 2)) + "_" + str(round(lat, 2))
        if geo_index not in rst:
            rst[geo_index] = {}
            rst[geo_index][1] = []
            rst[geo_index][0] = []
            rst[geo_index]['other'] = []
            rst[geo_index]['establish'] = 1 # establish
            rst[geo_index]['industry'] = [indu_code]
            rst[geo_index][oper_dummy].append(indu_code)
                
        else:
            rst[geo_index]['establish'] += 1
            rst[geo_index]['industry'].append(indu_code)
            rst[geo_index][oper_dummy].append(indu_code)
    
    stat = []
    for idx in rst:
        lon, lat = idx.split('_')
        establish_cnt = rst[idx]['establish']
        indu_cnt = Counter(rst[idx]['industry'])
        existence_cnt = len(rst[idx][1])
        exit_cnt = len(rst[idx][0])
        other_cnt = len(rst[idx]['other'])
        existence_indu_cnt = Counter(rst[idx][1])
        exit_indu_cnt = Counter(rst[idx][0])
                
        stat.append([idx, lon, lat, esta_year, establish_cnt, indu_cnt[1], indu_cnt[2], indu_cnt[3], 
                     existence_cnt, existence_indu_cnt[1], existence_indu_cnt[2], existence_indu_cnt[3],
                     exit_cnt, exit_indu_cnt[1], exit_indu_cnt[2], exit_indu_cnt[3], other_cnt])
    df = pd.DataFrame(stat, columns = ['idx', 'lon', 'lat', 'date', 
                                       'establish_total', 'establish_prim', 'establish_seco', 'establish_tert',
                                       'exist_total', 'exist_prim', 'exist_seco', 'exist_tert',
                                       'close_total', 'close_prim', 'close_seco', 'close_tert', 'other_total'])
    df.to_csv('GFRD_' + str(filename) + 'v2.csv', index = False, header = False)

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


In [71]:
# calculate the cancelled records
dt = pd.read_csv('firm_2005.csv', sep = '\t')
dt['approved_time'] = dt['approved_time'].str.slice(0, 4)

for k, v in dt.groupby('approved_time'):
    cnt = 0
    print k, len(v)
    for i in range(len(v)):
        stat = str(v.iloc[i,3])
        if any(s in stat for s in stat_exit):
            cnt += 1
    print cnt